# Interactable Test Notebook for Spectrogram Helper Functions

This notebook demonstrates how to update, create, and interactively run tests for the spectrogram plotting helper functions. It follows the code style and documentation conventions used in the main spectrogram scripts.

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
from pathlib import Path
import types
import pytest

# For demonstration, plotting libraries can be imported if needed
#import matplotlib.pyplot as plt

# Set up any global plotting style if needed (following spectrogram scripts)
# plt.style.use('seaborn-darkgrid')

## Define Functions to be Tested

Below are the helper functions to be tested. Each function includes a descriptive docstring in numpy format and is well commented, following the style of the spectrogram plotting scripts.

In [ ]:
# Helper function definitions for testing

def get_cdf_file_type(filename):
    """
    Extract the CDF file type from the filename.

    Parameters
    ----------
    filename : str
        The name of the CDF file.

    Returns
    -------
    str or None
        The file type string if found, otherwise None.
    """
    import re
    match = re.search(r'_([a-z]{3})_', filename)
    if match:
        return match.group(1)
    if 'orb' in filename:
        return 'orb'
    return None


def get_variable_shape(cdf_path, var_name):
    """
    Get the shape of a variable in a CDF file.

    Parameters
    ----------
    cdf_path : str
        Path to the CDF file.
    var_name : str
        Name of the variable to check.

    Returns
    -------
    tuple or None
        Shape of the variable, or None if not found.
    """
    # This is a mock implementation for testing
    class DummyCDF:
        def __init__(self, path):
            self.path = path
        def __enter__(self):
            return self
        def __exit__(self, exc_type, exc, tb):
            return False
        def varget(self, name):
            if name == 'ok':
                return np.zeros((2, 3))
            raise ValueError('Variable not found')
    try:
        with DummyCDF(cdf_path) as cdf:
            return cdf.varget(var_name).shape
    except ValueError as variable_not_found_error:
        # Return None if variable is not found
        return None


def get_timestamps_for_orbit(df, orbit, var_type, times):
    """
    Get timestamps for a given orbit from a DataFrame and time array.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing orbit and index information.
    orbit : int
        Orbit number to extract.
    var_type : str
        Variable type (e.g., 'ees').
    times : np.ndarray
        Array of timestamps.

    Returns
    -------
    list
        List of timestamps for the specified orbit.
    """
    row = df[df['Orbit'] == orbit]
    if row.empty:
        raise ValueError(f'Orbit {orbit} not found in DataFrame')
    min_idx = row[f'{var_type.upper()} Min Index'].values[0]
    max_idx = row[f'{var_type.upper()} Max Index'].values[0]
    return list(times[min_idx:max_idx+1])


def get_cdf_var_shapes(folder, var_names):
    """
    Get the shapes of variables in all CDF files in a folder.

    Parameters
    ----------
    folder : str
        Path to the folder containing CDF files.
    var_names : list of str
        List of variable names to check.

    Returns
    -------
    dict
        Dictionary mapping variable names to lists of shapes.
    """
    out = {v: [] for v in var_names}
    for file in Path(folder).glob('*.cdf'):
        for v in var_names:
            shape = get_variable_shape(str(file), v)
            out[v].append(shape)
    return out

## Update Existing Test Files

When updating test files, ensure that exception names are descriptive, docstrings are present and follow the numpy format, and comments match the style of the spectrogram plotting scripts.

## Create New Test Files

To add new tests, follow the same style: use descriptive exception names, numpy-style docstrings, and clear comments. This ensures maintainability and clarity.

## Create Python Notebook Versions

Convert your test files into Python notebooks to make them interactable. Place all function definitions in one cell, and create independent cells for each test. This allows for easy experimentation and debugging.

## Notebook: Function Definitions Cell

All function definitions are placed in a single cell above. You can modify and rerun this cell to update the functions used in the tests below.

## Notebook: Independent Test Cells

Each of the following cells runs a single test. You can run them independently to verify the behavior of each function. Exception names are descriptive, outputs are clear, and code style matches the spectrogram plotting scripts.

In [ ]:
# Test: get_cdf_file_type
try:
    assert get_cdf_file_type('foo_ees_bar.cdf') == 'ees', "Failed to detect 'ees' type"
    assert get_cdf_file_type('something_orb_data.cdf') == 'orb', "Failed to detect 'orb' type"
    assert get_cdf_file_type('no_match.cdf') is None, "Failed to return None for no match"
    print("test_get_cdf_file_type: PASSED")
except AssertionError as assertion_error:
    print(f"test_get_cdf_file_type: FAILED\n{assertion_error}")

In [ ]:
# Test: get_variable_shape
try:
    shape = get_variable_shape('dummy.cdf', 'ok')
    assert shape == (2, 3), f"Expected shape (2, 3), got {shape}"
    shape_none = get_variable_shape('dummy.cdf', 'missing')
    assert shape_none is None, "Expected None for missing variable"
    print("test_get_variable_shape: PASSED")
except AssertionError as assertion_error:
    print(f"test_get_variable_shape: FAILED\n{assertion_error}")
except Exception as unexpected_error:
    print(f"test_get_variable_shape: ERROR\n{unexpected_error}")

In [ ]:
# Test: get_timestamps_for_orbit
try:
    df = pd.DataFrame({
        'Orbit': [10, 11],
        'EES Min Index': [0, 2],
        'EES Max Index': [1, 4],
    })
    times = np.array([100.0, 200.0, 300.0, 400.0, 500.0])
    res = get_timestamps_for_orbit(df, 10, 'ees', times)
    assert res == [100.0, 200.0], f"Expected [100.0, 200.0], got {res}"
    # second orbit single point
    df.loc[1, 'EES Max Index'] = 2
    res2 = get_timestamps_for_orbit(df, 11, 'ees', times)
    assert res2 == [300.0], f"Expected [300.0], got {res2}"
    print("test_get_timestamps_for_orbit: PASSED")
except AssertionError as assertion_error:
    print(f"test_get_timestamps_for_orbit: FAILED\n{assertion_error}")
except Exception as unexpected_error:
    print(f"test_get_timestamps_for_orbit: ERROR\n{unexpected_error}")

In [ ]:
# Test: get_cdf_var_shapes
try:
    # Patch get_variable_shape to always return (1,)
    def patched_get_variable_shape(path, v):
        return (1,)
    original_get_variable_shape = get_variable_shape
    get_variable_shape = patched_get_variable_shape
    # Create dummy file list in a temp folder
    import tempfile
    import os
    with tempfile.TemporaryDirectory() as folder:
        open(os.path.join(folder, 'a.cdf'), 'w').close()
        open(os.path.join(folder, 'b.cdf'), 'w').close()
        out = get_cdf_var_shapes(folder, ['ok'])
        assert 'ok' in out and len(out['ok']) == 2, f"Expected 2 shapes, got {out}"
        assert all(s == (1,) for s in out['ok']), f"Expected all shapes (1,), got {out['ok']}"
    get_variable_shape = original_get_variable_shape
    print("test_get_cdf_var_shapes: PASSED")
except AssertionError as assertion_error:
    print(f"test_get_cdf_var_shapes: FAILED\n{assertion_error}")
except Exception as unexpected_error:
    print(f"test_get_cdf_var_shapes: ERROR\n{unexpected_error}")